<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from Akvo import Flow
from selenium import webdriver
from time import sleep
import os.path
import pandas as pd
import urllib.parse
import requests as r
import json

In [2]:
instanceURL = "https://anu.akvolumen.org/api"
workdir = "../charts-generator/"
jsondir = "new-json/"
cachedir = "lumen-cache/"
server = "http://localhost:4000/"

In [3]:
token = Flow.getToken()

In [4]:
dboard = Flow.getData(instanceURL + '/library', token)
visual = dboard.get('visualisations')
df = pd.DataFrame(visual)
#sl = pd.read_csv('list-of-figures-aligned-source.csv')
sl = pd.read_csv('list-of-images-with-lumen-2019-11-30.csv')
sl['Fid'] = sl['Fid'].apply(lambda x: x.replace('-','.'))
cm = sl[sl['source'] == 'Custom']
sl = sl[sl['source'] != 'Custom']

In [5]:
df['specstr'] = df['spec'].astype(str)

In [6]:
def parseString(x):
    x = x.replace("""'""","""\"""").replace('None','null').replace('False','false').replace('True','true')
    x = x.replace(' : ',':').replace(': ',':').replace(' :',':')
    x = x.replace('Don"t',"Don't")
    x = urllib.parse.unquote(x)
    x = urllib.parse.quote(x)
    return x

In [7]:
df['specurls'] = df['specstr'].apply(parseString)
#df['endpoints'] = '/aggregation/' + df['datasetId'] + '/' + df['visualisationType']
df['endpoints'] = '/aggregation/' + df['datasetId'] + '/' + df['visualisationType'] + '?query=' + df['specurls']

In [8]:
df = df.merge(sl, left_on='id', right_on='source')

In [9]:
ds = df[['Fid','Fname','endpoints','visualisationType','id','specstr']]

In [10]:
ds = ds.rename(columns={
    'Fid':'id',
    'Fname':'name',
    'endpoints':'endpoint',
    'visualisationType':'type',
    'id':'lumenid',
    'specstr':'parameter'
})

In [11]:
def renameProvince(x):
    renamedLabel = ''
    if x == "Maringe Kokota":
        renamedLabel = "Maringe / Kokota"
    elif x == "Gao Bugotu":
        renamedLabel = "Gao / Bugotu"
    elif x == "Russell Savo":
        renamedLabel = "Russells / Savo"
    elif x == "Lau Mbaelelea":
        renamedLabel = "Lau / Mbaelelea"
    elif x == "Aoke Langalanga":
        renamedLabel = "Aoke / Langalanga"
    else:
        renamedLabel = x
    return renamedLabel

In [12]:
getLegend = lambda x : [y['key'] for y in x]
getLabel = lambda x : [y['label'] for y in x]
getData = lambda x : [y['value'] for y in x]
ordered_province = pd.DataFrame([
    {'ordered':'Central Makira'},
    {'ordered':'East Guadalcanal'},
    {'ordered':'Central Guadalcanal'},
    {'ordered':'East Honiara'},
    {'ordered':'Central Honiara'},
    {'ordered':'West Honiara'},
    {'ordered':'Aoke Langalanga'},
    {'ordered':'Small Malaita'},
    {'ordered':'Lau Mbaelelea'},
    {'ordered':'Russell Savo'},
    {'ordered':'Gao Bugotu'},
    {'ordered':'Maringe Kokota'},
    {'ordered':'Gizo Kolombangara'},
    {'ordered':'North Vella Lavella'},
    {'ordered':'North West Choiseul'}
])
ordered_province['renamed'] = ordered_province['ordered'].apply(lambda x:renameProvince(x))
ordered_province.reset_index(level=0, inplace=True)

In [13]:
def checkPercentage(allValues):
    index = 0
    diff = 0
    total = sum(allValues)
    if total < 100:
        index = allValues.index(min(allValues))
        diff = 100 - total
    elif sum(allValues) > 100:
        index = allValues.index(max(allValues))
        diff = 100 - total
    else:
        pass
    return [index,diff]

In [14]:
yesno = ['Yes','No']
def checkColorOrder(dataSeries, dataCommon, graphType, legends):
    opt = ['Yes', 'No']
    if set(['Not Sure']).issubset(legends):
        opt.insert(2, 'Not Sure')
    legend_df = pd.Series(legends).to_frame('legend').reset_index()
    opt_df = pd.Series(opt).to_frame('order').reset_index()
    opt_df = opt_df.merge(legend_df, left_on='order', right_on='legend', suffixes=['_a', '_b'], how='outer')
    ordered_legends = list(opt_df.sort_values(by='index_a')['index_b'])
    ordered_series = []
    ordered_keys = []
    for i in ordered_legends:
        if graphType == 'bar':
            ordered_series.append(dataSeries[i])
        else:
            ordered_series.append(dataSeries[0]['data'][i])
            ordered_keys.append(dataCommon['data'][i])
    transformed = {
        "series": ordered_series,
        "keys": ordered_keys
    }
    return transformed

In [15]:
def createBarChart(b,ids):
    ordered = ordered_province
    data = b
    output = r.get(server + 'draft/results-barchart.json').json()
    unordered = pd.DataFrame(data['common']['data'])
    unordered.reset_index(level=0, inplace=True)
    ordered_index = unordered.merge(ordered,left_on='key',right_on='ordered')
    ordered_index = ordered_index.sort_values(by=['index_x'], ascending=True)
    available_province = ordered_index[['ordered','index_y']]
    available_province['ordered'] = available_province['ordered'].apply(renameProvince)
    available_province = list(available_province.sort_values(by=['index_y'], ascending=True)['ordered'])
    ordered_index = list(ordered_index['index_y'])
    legends = getLegend(data['series'])
    if set(yesno).issubset(legends):
        transformed = checkColorOrder(data['series'], None, 'bar', legends)
        data['series'] = transformed['series']
    output['legend']['data'] = getLegend(data['series'])
    output['xAxis']['data'] = available_province
    for i,l in enumerate(output['legend']['data']):
        if i == 0:
            df = pd.DataFrame(data['series'][i]['data']).rename(columns={'value':i})
        else:
            df[i] = pd.DataFrame(data['series'][i]['data']).rename(columns={'value':i})[i]
    df['sum'] = df.sum(axis = 1, skipna = True)
    df['order'] = pd.Series(ordered_index)
    df = df.sort_values(by=['order'], ascending=True).reset_index()
    for i,l in enumerate(output['legend']['data']):
        df[i] = round(df[i] / df['sum'] * 100).astype(int)
    df = df.drop(columns=['sum'])
    series = list(range(len(output["legend"]["data"])))
    for i, rows in df.iterrows():
        new_list = []
        for a in series:
            new_list.append(rows[a])
        adjust = checkPercentage(new_list)
        df[adjust[0]][i] = df[adjust[0]][i] + adjust[1]
    output['series'] = []
    for i,l in enumerate(output["legend"]["data"]):
        dataList = [None if di is 0 else di for di in list(df[i])]
        print(dataList)
        output["series"].append({"name": l,
                                 "type": "bar",
                                 "stack": "total",
                                 "label": {
                                     "normal": {
                                         "fontSize": 20,
                                         "fontWeight": "bold",
                                         "show": True,
                                         "position": "inside",
                                         "formatter": "{c}%"
                                     }
                                },"data": dataList})
    json_output = json.dumps(output)
    jname = ids.replace(".","-")
    f = open(workdir + jsondir + jname +".json","w")
    f.write(json_output)
    f.close()
    return jname

In [16]:
def createPieChart(b,ids):
    data = b
    output = r.get(server + 'draft/results-piechart.json').json()
    legends = getLegend(data['common']['data'])
    if set(yesno).issubset(legends):
        transformed = checkColorOrder(data['series'], data['common'],'pie', legends)
        data['series'][0]['data'] = transformed['series']
        data['common']['data'] = transformed['keys']
    ## Total
    ## output['title']['text'] = str(sum(getData(data['series'][0]['data'])))
    output['series'][0]['data'] = []
    for i,d in enumerate(data['series'][0]['data']):
        output['series'][0]['data'].append({'value':d['value'],'name':data['common']['data'][i]['key']})
    getName = lambda x : [y['name'] for y in x]
    output['legend']['data'] = getName(output['series'][0]['data'])
    json_output = json.dumps(output)
    jname = ids.replace(".","-")
    f = open(workdir + jsondir + jname +".json","w")
    f.write(json_output)
    f.close()
    return jname

In [17]:
def generateChart(endpoint,charttype,ids,lumenid,index):
    cachename = lumenid + '.json'
    jsoncache = workdir + cachedir + cachename
    if os.path.exists(jsoncache):
        data = r.get(server + cachedir + cachename).json()
    else:
        data = Flow.getData(instanceURL + endpoint, token)
        f = open(jsoncache,"w")
        f.write(json.dumps(data))
        f.close()
    if charttype == 'pie':
        result = createPieChart(data,ids)
    else:
        result = createBarChart(data,ids)
    return result

In [18]:
ds['order'] = ds['id'].apply(lambda x: x.replace('.','.0') if len(x) == 3 else x)
ds = ds.sort_values('order')
ds.reset_index(level=0,inplace=True)

In [19]:
ds

,index,id,name,endpoint,type,lumenid,parameter,order
0,20,3.3,Voters who checked their name at a voter regis...,/aggregation/5c980e26-be25-47ee-9fe1-13eb6bdc2...,bar,5ccfb75d-da71-43a8-b022-cb302e96d72b,"{'filters': [], 'sort': None, 'subBucketColumn...",3.03
1,47,3.4,Citizens who were asked to register in another...,/aggregation/5c980e26-be25-47ee-9fe1-13eb6bdc2...,bar,5cd41506-bf3d-4c6f-a2d0-77a5fc77c145,"{'filters': [], 'sort': None, 'subBucketColumn...",3.04
2,26,3.6,Citizens who changed their registration to ano...,/aggregation/5c91ad95-f929-41dd-bbec-1436d56e9...,bar,5ccfbb2d-b044-47bc-b980-709d07d72cb6,"{'filters': [], 'sort': None, 'subBucketColumn...",3.06
3,41,3.7,Citizens who know of voters who registered in ...,/aggregation/5c980e26-be25-47ee-9fe1-13eb6bdc2...,bar,5cd414ca-882e-4ef0-9530-6a306e4c9649,"{'filters': [], 'sort': None, 'subBucketColumn...",3.07
4,46,3.8,Voters who thought the electoral roll in their...,/aggregation/5c91ad95-f929-41dd-bbec-1436d56e9...,bar,5c9c374a-8d58-4838-9ca5-c8f404015f12,"{'filters': [], 'sort': None, 'subBucketColumn...",3.08
5,5,3.10,Citizens who saw any SIEC media or awareness (...,/aggregation/5c980e26-be25-47ee-9fe1-13eb6bdc2...,bar,5c99bbe4-d49a-47ef-a143-4e40994adc40,"{'filters': [], 'sort': None, 'subBucketColumn...",3.10
6,44,3.13,Voters who discussed the election on social me...,/aggregation/5c980e26-be25-47ee-9fe1-13eb6bdc2...,bar,5cad913a-4038-4f56-9478-25da8fe0727a,"{'filters': [], 'sort': None, 'subBucketColumn...",3.13
7,50,3.14,Voters who discussed the election on social me...,/aggregation/5c980e26-be25-47ee-9fe1-13eb6bdc2...,bar,5d0acf80-c117-4647-88b7-5d4f1cac79aa,"{'filters': [], 'sort': None, 'subBucketColumn...",3.14
8,30,3.16,Election-related tension or violence observed ...,/aggregation/5c9c15de-864a-49a6-931a-5b081d33d...,bar,5ccfc09c-f946-4cb2-b611-d3aee2d17d92,"{'filters': [], 'sort': None, 'subBucketColumn...",3.16
9,29,3.17,Observers who heard about election-related ten...,/aggregation/5c9c15de-864a-49a6-931a-5b081d33d...,bar,5ccfc1f9-c780-4451-ba9f-83ba5262a054,"{'filters': [], 'sort': None, 'subBucketColumn...",3.17


In [20]:
ds['json'] =  ds.apply(lambda x: generateChart(x['endpoint'],x['type'],str(x['id']),x['lumenid'],x['id']), axis=1)

/Users/dedenbangkit/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


[90, 87, 97, 70, 79, 69, 82, 88, 93, 96, 98, 98, 82, 88, 98]
[10, 13, 3, 30, 21, 31, 18, 12, 7, 4, 2, 2, 18, 12, 2]
[3, 11, 11, 42, 26, 29, 22, 13, 25, 10, 19, 9, 15, 9, 7]
[97, 89, 89, 58, 74, 71, 78, 87, 75, 90, 81, 91, 85, 91, 93]
[8, 8, 12, 18, 10, 30, 6, 10, 17, 6, 11, 5, 18, 10, 2]
[91, 92, 86, 79, 89, 70, 94, 89, 83, 93, 89, 94, 81, 90, 98]
[1, None, 2, 3, 1, None, None, 1, None, 1, None, 1, 1, None, None]
[23, 32, 28, 39, 39, 28, 40, 15, 38, 32, 37, 56, 51, 54, 38]
[77, 68, 72, 61, 61, 72, 60, 85, 62, 68, 63, 44, 49, 46, 62]
[63, 58, 82, 30, 44, 51, 51, 48, 70, 32, 68, 75, 48, 65, 91]
[12, 35, 7, 45, 37, 34, 37, 32, 25, 54, 15, 18, 43, 22, 6]
[25, 7, 11, 25, 19, 15, 12, 20, 5, 14, 17, 7, 9, 13, 3]
[51, 56, 77, 87, 83, 85, 84, 80, 72, 66, 75, 59, 79, 80, 27]
[49, 44, 23, 13, 17, 15, 16, 20, 28, 34, 25, 41, 21, 20, 73]
[3, 4, 7, 24, 21, 17, 10, 8, 13, 2, 8, 2, 21, 2, 1]
[97, 96, 93, 76, 79, 83, 90, 92, 87, 98, 92, 98, 79, 98, 99]


/Users/dedenbangkit/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[19, 9, 2]
[80, 90, 96]
[1, None, None]
[None, 1, 2]
[None, None, None, None, 20, None, None, None, None, None, None, None, 20, 40, None]
[100, 100, 100, 67, 80, 100, 100, 100, 100, 100, 100, 100, 80, 60, 100]
[None, None, None, 33, None, None, None, None, None, None, None, None, None, None, None]
[None, 34, None, 17, 20, None, 40, 40, 40, 83, 40, 40, 20, 100, 80]
[100, 33, 100, 33, 80, 100, 60, 60, 60, 17, 60, 60, 80, None, 20]
[None, 33, None, 50, None, None, None, None, None, None, None, None, None, None, None]
[74, 66, 77, 77, 86, 85, 77, 67, 69, 79, 59, 74, 81, 71, 64]
[12, 9, 18, 12, 7, 10, 14, 26, 20, 6, 29, 17, 15, 19, 16]
[14, 25, 5, 11, 7, 5, 9, 7, 11, 15, 12, 9, 4, 10, 20]
[74, 75]
[13, 16]
[13, 9]
[41, 81, 43, 52, 51, 49, 32, 47, 67, 71, 50, 47, 32, 69, 22]
[20, 7, 45, 29, 30, 24, 54, 26, 18, 11, 39, 40, 15, 19, 62]
[39, 12, 12, 19, 19, 27, 14, 27, 15, 18, 11, 13, 53, 12, 16]
[50, 52]
[27, 31]
[23, 17]
[16, 13, 35, 25, 38, 25, 16, 7, 54, 39, 43, 13, 23, 24, 16]
[84, 87, 65,

In [21]:
def takeScreenShot(x,y):
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument("--test-type")
    options.add_argument("--window-size=1920,1080")
    driver = webdriver.Chrome(chrome_options=options)
    driver.get('http://localhost:4000/#'+x)
    sleep(3)
    imagename = x + '-' + y.lower().replace('?',' ').replace('(','').replace(')','').replace(' ','-').replace('/','-')
    driver.save_screenshot('screenshots-1202/'+ imagename +'.png')
    driver.close()
    return imagename + '.png'

In [22]:
for row in ds.to_dict('records'):
    takeScreenShot(row['json'],row['name'])

/Users/dedenbangkit/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  
/Users/dedenbangkit/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  
/Users/dedenbangkit/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  
/Users/dedenbangkit/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  
/Users/dedenbangkit/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  
/Users/dedenbangkit/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  
/Users/dedenbangkit/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chr

In [23]:
#ds = ds[ds['type'] == 'bar']

In [24]:
#ds.append({'id',})

In [25]:
#for retake in retakes:
#    takeScreenShot(retake.replace('.','-'),retake.replace('.','-')+'_new-')

In [33]:
ds

,index,id,name,endpoint,type,lumenid,parameter,order,json
0,20,3.3,Voters who checked their name at a voter regis...,/aggregation/5c980e26-be25-47ee-9fe1-13eb6bdc2...,bar,5ccfb75d-da71-43a8-b022-cb302e96d72b,"{'filters': [], 'sort': None, 'subBucketColumn...",3.03,3-3
1,47,3.4,Citizens who were asked to register in another...,/aggregation/5c980e26-be25-47ee-9fe1-13eb6bdc2...,bar,5cd41506-bf3d-4c6f-a2d0-77a5fc77c145,"{'filters': [], 'sort': None, 'subBucketColumn...",3.04,3-4
2,26,3.6,Citizens who changed their registration to ano...,/aggregation/5c91ad95-f929-41dd-bbec-1436d56e9...,bar,5ccfbb2d-b044-47bc-b980-709d07d72cb6,"{'filters': [], 'sort': None, 'subBucketColumn...",3.06,3-6
3,41,3.7,Citizens who know of voters who registered in ...,/aggregation/5c980e26-be25-47ee-9fe1-13eb6bdc2...,bar,5cd414ca-882e-4ef0-9530-6a306e4c9649,"{'filters': [], 'sort': None, 'subBucketColumn...",3.07,3-7
4,46,3.8,Voters who thought the electoral roll in their...,/aggregation/5c91ad95-f929-41dd-bbec-1436d56e9...,bar,5c9c374a-8d58-4838-9ca5-c8f404015f12,"{'filters': [], 'sort': None, 'subBucketColumn...",3.08,3-8
5,5,3.10,Citizens who saw any SIEC media or awareness (...,/aggregation/5c980e26-be25-47ee-9fe1-13eb6bdc2...,bar,5c99bbe4-d49a-47ef-a143-4e40994adc40,"{'filters': [], 'sort': None, 'subBucketColumn...",3.10,3-10
6,44,3.13,Voters who discussed the election on social me...,/aggregation/5c980e26-be25-47ee-9fe1-13eb6bdc2...,bar,5cad913a-4038-4f56-9478-25da8fe0727a,"{'filters': [], 'sort': None, 'subBucketColumn...",3.13,3-13
7,50,3.14,Voters who discussed the election on social me...,/aggregation/5c980e26-be25-47ee-9fe1-13eb6bdc2...,bar,5d0acf80-c117-4647-88b7-5d4f1cac79aa,"{'filters': [], 'sort': None, 'subBucketColumn...",3.14,3-14
8,30,3.16,Election-related tension or violence observed ...,/aggregation/5c9c15de-864a-49a6-931a-5b081d33d...,bar,5ccfc09c-f946-4cb2-b611-d3aee2d17d92,"{'filters': [], 'sort': None, 'subBucketColumn...",3.16,3-16
9,29,3.17,Observers who heard about election-related ten...,/aggregation/5c9c15de-864a-49a6-931a-5b081d33d...,bar,5ccfc1f9-c780-4451-ba9f-83ba5262a054,"{'filters': [], 'sort': None, 'subBucketColumn...",3.17,3-17


In [36]:
ds[ds['id'] == '5.9']['name'].values

array(['Citizens perceptions of who influenced their vote (by constituency)'],
      dtype=object)

In [ ]:
recapture = [
    {'j':'4-17','n':'Voters who thought CDFs improved development in their constituency (by constituency)'},
    {'j':'4-19','n':'Voters who said they or their family had received CDF support in the last five years (by constituency)'},
    {'j':'5-8','n':'Citizens perceptions of who influenced their vote (all constituencies)'},
    {'j':'5-9','n':'Citizens perceptions of who influenced their vote (by constituency)'}
]

In [26]:
#takeScreenShot('3-3',row['name'])